# Make mass catalogs for objects using spectroscopic redshifts

In [ ]:
from pathlib import Path
from importlib import reload
import joblib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.lines as lines
from astropy.table import Table, vstack, join

import mpl_scatter_density
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

from desitarget.sv1.sv1_targetmask import desi_mask, bgs_mask
import utils
reload(utils)

# Create BGS Catalog

In [ ]:
my_path = Path("/global/cscratch1/sd/bid13/LRG")
bgs_path_n = my_path / ("BGS_SV_north.fits")
bgs_path_s = my_path / ("BGS_SV_south.fits")

In [ ]:
bgs_sv_cat = vstack([Table.read(bgs_path_n), Table.read(bgs_path_s)]) # read in the SV data
names = [name for name in bgs_sv_cat.colnames if len(bgs_sv_cat[name].shape) <= 1]
bgs_sv_cat = bgs_sv_cat[names].to_pandas()

In [ ]:
# z_cat = Table.read("/global/cfs/cdirs/desi/survey/catalogs/SV1/redshift_comps/cascades/3.1/BGS_ANY/alltiles_BGS_ANYzinfo.fits")
# names = [name for name in z_cat.colnames if len(z_cat[name].shape) <= 1]
# z_cat = z_cat[names].to_pandas()
z_cat = Table.read("/global/cfs/cdirs/desi/survey/catalogs/SV1/redshift_comps/denali/testdenali/All/alltiles_Allzinfo.fits")
z_cat = z_cat[(z_cat["SV1_DESI_TARGET"] & desi_mask.mask("BGS_ANY"))>0]
names = [name for name in z_cat.colnames if len(z_cat[name].shape) <= 1]
z_cat = z_cat[names].to_pandas()

In [ ]:
z_cat["logmass"] = -99

In [ ]:
z_cat_merged = z_cat.merge(bgs_sv_cat, on="TARGETID", how="left")

In [ ]:
regrf_n = joblib.load(f'/global/cfs/cdirs/desi/users/rongpu/ls_dr9.0_photoz/stellar_mass/rf/rf_20210206_north.pkl')
regrf_s = joblib.load(f'/global/cfs/cdirs/desi/users/rongpu/ls_dr9.0_photoz/stellar_mass/rf/rf_20210206_south.pkl')
# # Get masses based on spec-z instead of photoz
# regrf_n = joblib.load(f'/global/cfs/cdirs/desi/users/rongpu/ls_dr9.0_photoz/stellar_mass/rf/rf_20210206_north-mgc_redshift.pkl')
# # Get masses based on spec-z instead of photoz
# regrf_s = joblib.load(f'/global/cfs/cdirs/desi/users/rongpu/ls_dr9.0_photoz/stellar_mass/rf/rf_20210206_south-mgc_redshift.pkl')

In [ ]:
good_mask = (np.isfinite(z_cat_merged["gmag"]) & np.isfinite(z_cat_merged["rmag"]) & np.isfinite(z_cat_merged["zmag"]))
good_mask &= (np.isfinite(z_cat_merged["w1mag"]) & np.isfinite(z_cat_merged["w2mag"]))
good_mask &= (z_cat_merged["FIBERSTATUS"]==0)
mask_north = (z_cat_merged["PHOTSYS_x"]==b"N")
mask_south = (z_cat_merged["PHOTSYS_x"]==b"S")

In [ ]:
#North
mask = good_mask & mask_north
data = np.column_stack((z_cat_merged['gmag']-z_cat_merged['rmag'], z_cat_merged['rmag']-z_cat_merged['zmag'], z_cat_merged['zmag']-z_cat_merged['w1mag'], z_cat_merged['w1mag']-z_cat_merged['w2mag'], z_cat_merged['Z']))
data = data[mask]
distmod = cosmo.distmod(z_cat_merged['Z'])
distmod = distmod[mask]
m_to_l_predict = regrf_n.predict(data)
z_cat.loc[mask,"logmass"] = m_to_l_predict + 0.4*(4.45-(z_cat_merged['zmag'][mask] - distmod.to_value() ))

In [ ]:
#South
mask = good_mask & mask_south
data = np.column_stack((z_cat_merged['gmag']-z_cat_merged['rmag'], z_cat_merged['rmag']-z_cat_merged['zmag'], z_cat_merged['zmag']-z_cat_merged['w1mag'], z_cat_merged['w1mag']-z_cat_merged['w2mag'], z_cat_merged['Z']))
data = data[mask]
distmod = cosmo.distmod(z_cat_merged['Z'])
distmod = distmod[mask]
m_to_l_predict = regrf_s.predict(data)
z_cat.loc[mask,"logmass"] = m_to_l_predict + 0.4*(4.45-(z_cat_merged['zmag'][mask] - distmod.to_value() ))

In [ ]:
# z_cat_fin = Table.read("/global/cfs/cdirs/desi/survey/catalogs/SV1/redshift_comps/cascades/3.1/BGS_ANY/alltiles_BGS_ANYzinfo.fits")
z_cat_fin = Table.read("/global/cfs/cdirs/desi/survey/catalogs/SV1/redshift_comps/denali/testdenali/All/alltiles_Allzinfo.fits")
z_cat_fin = z_cat_fin[(z_cat_fin["SV1_DESI_TARGET"] & desi_mask.mask("BGS_ANY"))>0]
z_cat_fin["logmass"] =z_cat["logmass"]

In [ ]:
z_cat_fin.write(my_path/"alltiles_BGS_ANYzinfo_stellar_mass_photo-z.fits")

# LRG

In [ ]:
my_path = Path("/global/cscratch1/sd/bid13/LRG")
lrg_path_n = my_path / ("LRG_SV_north.fits")
lrg_path_s = my_path / ("LRG_SV_south.fits")

In [ ]:
lrg_sv_cat = vstack([Table.read(lrg_path_n), Table.read(lrg_path_s)]) # read in the SV data
names = [name for name in lrg_sv_cat.colnames if len(lrg_sv_cat[name].shape) <= 1]
lrg_sv_cat = lrg_sv_cat[names].to_pandas()

In [ ]:
# z_cat = Table.read("/global/cfs/cdirs/desi/survey/catalogs/SV1/redshift_comps/cascades/3.1/LRG/alltiles_LRGzinfo.fits")
# names = [name for name in z_cat.colnames if len(z_cat[name].shape) <= 1]
# z_cat = z_cat[names].to_pandas()

# z_cat = Table.read("/global/cfs/cdirs/desi/survey/catalogs/SV1/redshift_comps/denali/testdenali/All/alltiles_Allzinfo.fits")
z_cat = Table.read("/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/LSScats/v0/LRGAlltiles_full.dat.fits")
# z_cat = z_cat[(z_cat["SV1_DESI_TARGET"] & desi_mask.mask("LRG"))>0]
names = [name for name in z_cat.colnames if len(z_cat[name].shape) <= 1]
z_cat = z_cat[names].to_pandas()

In [ ]:
z_cat["logmass"] = -99

In [ ]:
z_cat_merged = z_cat.merge(lrg_sv_cat, on="TARGETID", how="left")

In [ ]:
# Get masses based on spec-z instead of photoz
regrf_n = joblib.load(f'/global/cfs/cdirs/desi/users/rongpu/ls_dr9.0_photoz/stellar_mass/rf/rf_20210206_north-mgc_redshift.pkl')
# Get masses based on spec-z instead of photoz
regrf_s = joblib.load(f'/global/cfs/cdirs/desi/users/rongpu/ls_dr9.0_photoz/stellar_mass/rf/rf_20210206_south-mgc_redshift.pkl')

In [ ]:
good_mask = (np.isfinite(z_cat_merged["gmag"]) & np.isfinite(z_cat_merged["rmag"]) & np.isfinite(z_cat_merged["zmag"]))
good_mask &= (np.isfinite(z_cat_merged["w1mag"]) & np.isfinite(z_cat_merged["w2mag"]))
good_mask &= (z_cat_merged["FIBERSTATUS"]==0)
mask_north = (z_cat_merged["PHOTSYS_x"]==b"N")
mask_south = (z_cat_merged["PHOTSYS_x"]==b"S")

In [ ]:
#North
mask = good_mask & mask_north
data = np.column_stack((z_cat_merged['gmag']-z_cat_merged['rmag'], z_cat_merged['rmag']-z_cat_merged['zmag'], z_cat_merged['zmag']-z_cat_merged['w1mag'], z_cat_merged['w1mag']-z_cat_merged['w2mag'], z_cat_merged['Z']))
data = data[mask]
distmod = cosmo.distmod(z_cat_merged['Z'])
distmod = distmod[mask]
m_to_l_predict = regrf_n.predict(data)
z_cat.loc[mask,"logmass"] = m_to_l_predict + 0.4*(4.45-(z_cat_merged['zmag'][mask] - distmod.to_value() ))

In [ ]:
#South
mask = good_mask & mask_south
data = np.column_stack((z_cat_merged['gmag']-z_cat_merged['rmag'], z_cat_merged['rmag']-z_cat_merged['zmag'], z_cat_merged['zmag']-z_cat_merged['w1mag'], z_cat_merged['w1mag']-z_cat_merged['w2mag'], z_cat_merged['Z']))
data = data[mask]
distmod = cosmo.distmod(z_cat_merged['Z'])
distmod = distmod[mask]
m_to_l_predict = regrf_s.predict(data)
z_cat.loc[mask,"logmass"] = m_to_l_predict + 0.4*(4.45-(z_cat_merged['zmag'][mask] - distmod.to_value() ))

In [ ]:
# z_cat_fin = Table.read("/global/cfs/cdirs/desi/survey/catalogs/SV1/redshift_comps/cascades/3.1/LRG/alltiles_LRGzinfo.fits")
z_cat_fin = Table.read("/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/LSScats/v0/LRGAlltiles_full.dat.fits")
# z_cat_fin = z_cat_fin[(z_cat_fin["SV1_DESI_TARGET"] & desi_mask.mask("LRG"))>0]
z_cat_fin["logmass"] =z_cat["logmass"]

In [ ]:
z_cat_fin.write(my_path/"LRGAlltiles_full_stellar_mass.fits")

# Secondary Targets

In [ ]:
import desispec.io

In [ ]:
cat = pd.read_csv("/global/cscratch1/sd/bid13/test/LowMassAGN.txt", delim_whitespace=True)

In [ ]:
Z = []
gmag =[]
rmag = []
zmag = []
w1mag = []
w2mag = []
PHOTSYS=[]
for i in range(len(cat)):
    z_temp = Table.read(cat["Path"][i]+cat["Zbest_Files"][i])
    z_temp = float(z_temp[z_temp["TARGETID"]==cat["#TARGETID"][i]]["Z"])
    Z.append(z_temp)
    fibm_temp = desispec.io.read_spectra((cat["Path"][i]+cat["Zbest_Files"][i]).replace("zbest","coadd")).fibermap
    f = 22.5 - 2.5 * np.log10(float(fibm_temp[fibm_temp["TARGETID"]==cat["#TARGETID"][i]]["FLUX_G"]))
    gmag.append(f)
    f = 22.5 - 2.5 * np.log10(float(fibm_temp[fibm_temp["TARGETID"]==cat["#TARGETID"][i]]["FLUX_R"]))
    rmag.append(f)
    f = 22.5 - 2.5 * np.log10(float(fibm_temp[fibm_temp["TARGETID"]==cat["#TARGETID"][i]]["FLUX_Z"]))
    zmag.append(f)
    f = 22.5 - 2.5 * np.log10(float(fibm_temp[fibm_temp["TARGETID"]==cat["#TARGETID"][i]]["FLUX_W1"]))
    w1mag.append(f)
    f = 22.5 - 2.5 * np.log10(float(fibm_temp[fibm_temp["TARGETID"]==cat["#TARGETID"][i]]["FLUX_W2"]))
    w2mag.append(f)
    f = fibm_temp[fibm_temp["TARGETID"]==cat["#TARGETID"][i]]["PHOTSYS"]
    PHOTSYS.append(f)
Z = np.array(Z)
gmag =np.array(gmag)
rmag = np.array(rmag)
zmag = np.array(zmag)
w1mag = np.array(w1mag)
w2mag = np.array(w2mag)
PHOTSYS= np.array(PHOTSYS)

In [ ]:
data = np.column_stack((gmag-rmag, rmag-zmag, zmag-w1mag, w1mag-w2mag, Z))

In [ ]:
mask = np.all(np.isfinite(data), axis=1)
mask_n = (PHOTSYS=="N")
mask_s = (PHOTSYS=="S")

In [ ]:
cat["logmass"] = -99

In [ ]:
PHOTSYS = PHOTSYS[:,0].astype(str)

In [ ]:
# Get masses based on spec-z instead of photoz
regrf_n = joblib.load(f'/global/cfs/cdirs/desi/users/rongpu/ls_dr9.0_photoz/stellar_mass/rf/rf_20210206_north-mgc_redshift.pkl')
# Get masses based on spec-z instead of photoz
regrf_s = joblib.load(f'/global/cfs/cdirs/desi/users/rongpu/ls_dr9.0_photoz/stellar_mass/rf/rf_20210206_south-mgc_redshift.pkl')

In [ ]:
cat = cat[mask]
data = data[mask]
Z=Z[mask]
cat["PHOTSYS"] = PHOTSYS[mask]
cat["Z"]=Z
zmag = zmag[mask]

In [ ]:
cat = cat.reset_index(drop=True)

In [ ]:
for i in range(len(cat)):
    distmod = cosmo.distmod(Z[i])
    if cat["PHOTSYS"][i] =="N":
        m_to_l_predict = regrf_n.predict(data[i][np.newaxis,:])
        cat.loc[i,"logmass"] = m_to_l_predict + 0.4*(4.45-(zmag[i] - distmod.to_value() ))
    if cat["PHOTSYS"][i] =="S":
        m_to_l_predict = regrf_s.predict(data[i][np.newaxis,:])
        cat.loc[i,"logmass"] = m_to_l_predict + 0.4*(4.45-(zmag[i] - distmod.to_value() ))

In [ ]:
cat.to_csv("/global/cscratch1/sd/bid13/test/LowMassAGN_mass.txt")